In [ ]:
!git clone https://github.com/Jacobiano/morpholayers.git

In [ ]:
import tensorflow
import numpy as np
import matplotlib.pyplot as plt

plt.gray()
plt.rc("font", family="serif")
plt.rc("xtick", labelsize="x-small")
plt.rc("ytick", labelsize="x-small")

import skimage

print("TensorFlow version:", tensorflow.__version__)
print("Numpy version:", np.__version__)
print("Skimage version:", skimage.__version__)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    MaxPooling2D,
    Conv2D,
    Flatten,
    Dropout,
    Dense,
)
from matplotlib import pyplot as plt
import numpy as np
from morpholayers.layers import (
    DepthwiseDilation2D,
)

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape)

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
use_samples = 5000

# the data, split between train and test sets
x_train = x_train[0:use_samples]
y_train = y_train[0:use_samples]

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Define the text labels
fashion_mnist_labels = [
    "T-shirt/top",  # index 0
    "Trouser",  # index 1
    "Pullover",  # index 2
    "Dress",  # index 3
    "Coat",  # index 4
    "Sandal",  # index 5
    "Shirt",  # index 6
    "Sneaker",  # index 7
    "Bag",  # index 8
    "Ankle boot",
]  # index 9

plt.figure(figsize=(16, 16))
for i in range(25):
    # Image index, you can pick any number between 0 and 59,999
    img_index = i + 1
    # y_train contains the lables, ranging from 0 to 9
    label_index = np.argmax(y_train, axis=1)[img_index]
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train[img_index, :, :, 0])
    plt.title((fashion_mnist_labels[label_index]))

In [ ]:
# import skimage.morphology as skm
# cross=skm.disk(1)
# ownSE=skm.disk(1)

In [ ]:
xin = Input(shape=input_shape)
x = Conv2D(25, padding="valid", kernel_size=(3, 3), activation="relu")(xin)
x = MaxPooling2D(pool_size=(2, 2), padding="same")(x)
x = Conv2D(25, padding="valid", kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2), padding="same")(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
xoutput = Dense(num_classes, activation="softmax")(x)
model = Model(xin, outputs=xoutput)

In [ ]:
tensorflow.keras.utils.plot_model(
    model, to_file="model.png", show_shapes=True, show_layer_names=False
)
plt.figure(figsize=(20, 15))
plt.imshow(plt.imread("model.png"))
plt.axis("off")
plt.title("Architecture")
plt.show()
print("Number of Parameters by Layer")
model.summary()

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=tensorflow.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(
    x_train, y_train, batch_size=24, epochs=10, validation_data=(x_test, y_test)
)

In [ ]:
model.layers

In [ ]:
model.layers[1].weights[0].shape

In [ ]:
xin = Input(shape=input_shape)
x = Conv2D(25, padding="valid", kernel_size=(3, 3), activation="relu")(xin)
x = DepthwiseDilation2D(kernel_size=(2, 2), strides=(2, 2), trainable=True)(x)
x = Conv2D(25, padding="valid", kernel_size=(3, 3), activation="relu")(x)
x = DepthwiseDilation2D(kernel_size=(2, 2), strides=(2, 2), trainable=True)(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
xoutput = Dense(num_classes, activation="softmax")(x)
modeltrainable = Model(xin, outputs=xoutput)

In [ ]:
modeltrainable.layers

In [ ]:
print("Initialization as MaxPooling!")
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(modeltrainable.layers[2].get_weights()[0][:, :, i, 0])
    plt.axis("off")

We copy the weights from convolutional and dense layers to the second model (modeltrainable)

In [ ]:
modeltrainable.layers[1].set_weights(model.layers[1].get_weights())
modeltrainable.layers[3].set_weights(model.layers[3].get_weights())
modeltrainable.layers[7].set_weights(model.layers[7].get_weights())

If you only wish to train SE in the dilation, you should uncomment the following lines.

In [ ]:
# modeltrainable.layers[1].trainable=False
# modeltrainable.layers[3].trainable=False
# modeltrainable.layers[7].trainable=False

In [ ]:
modeltrainable.compile(
    loss="categorical_crossentropy",
    optimizer=tensorflow.keras.optimizers.Adam(),
    metrics=["accuracy"],
)
historytrainable = modeltrainable.fit(
    x_train, y_train, batch_size=24, epochs=20, validation_data=(x_test, y_test)
)

In [ ]:
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(modeltrainable.layers[2].get_weights()[0][:, :, i, 0])
    plt.axis("off")